In [4]:
import geopandas as gpd
import numpy as np
import pandas as pd
from owslib.wms import WebMapService
import matplotlib.pyplot as plt
import shapely    # if unable pip install Shapely
from datetime import datetime
import time
import datetime as dt
import os
from geolocation import GeoLocation  #You need to have geolocation.py in the same directory
import cv2    #If uninstalled do pip install opencv-python

In [87]:
#os.getcwd()

### Using Downloaded Data

In [5]:
fire19 = gpd.read_file('2019_perimeters_dd83.shp') #downloaded from GeoMACwebsite
fire18 = gpd.read_file('2018_perimeters_dd83.shp')
fire17 = gpd.read_file('2017_perimeters_dd83.shp')
fire16 = gpd.read_file('2016_perimeters_dd83.shp')

In [6]:
frames = [fire19, fire18, fire17, fire16]
#all_fires = pd.concat(frames)
all_fires = gpd.GeoDataFrame( pd.concat( frames, ignore_index=True) )
                    

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
len(all_fires)

20946

#### Downloading for single image

In [8]:
def getAutomatedBbox(centroid_lon, centroid_lat, distance):
    loc = GeoLocation.from_degrees(centroid_lat, centroid_lon)
    #distance = 1 
    (SW_loc, NE_loc) = loc.bounding_locations(distance)
    min_lat =  SW_loc.deg_lat
    min_lon =  SW_loc.deg_lon
    max_lat =  NE_loc.deg_lat
    max_lon =  NE_loc.deg_lon
    Bbox = (min_lon, min_lat, max_lon, max_lat)
    
    return Bbox

In [9]:
#Sample use
i = 9
distance = 0.5
if type(all_fires['geometry'][i]) == shapely.geometry.polygon.Polygon:
    test_polygon = all_fires['geometry'][i]
    ade_center = np.array(test_polygon.representative_point()) 
    polybbox = getAutomatedBbox(ade_center[0], ade_center[1], distance)
elif type(all_fires['geometry'][i]) == shapely.geometry.multipolygon.MultiPolygon:
    test_polygon = all_fires['geometry'][i][0]
    ade_center = np.array(test_polygon.representative_point()) 
    polybbox = getAutomatedBbox(ade_center[0], ade_center[1], distance)


In [144]:
wms = WebMapService('http://services.sentinel-hub.com/ogc/wms/259ecbcc-6ac9-424a-ad00-55ae902cd1df')

In [11]:
#get the same image using centroid approach
fire_imgnew = wms.getmap(  
                    layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2019-05-05/2019-05-05',        #may 23 (05, 24) to july 1st>             
                     )

decoded = cv2.imdecode(np.frombuffer(fire_imgnew.read(), np.uint8), -1)

## Downloading Sentinel Images to Match Perimeter Locations

Sentinel images are pictures of plots of land from the Sentinel Satelite with multiple types of features (vegetation, etc.). The images are taken every 5 days.

In [141]:
wms = WebMapService('http://services.sentinel-hub.com/ogc/wms/259ecbcc-6ac9-424a-ad00-55ae902cd1df')

### Preliminary exploration of WMS options/features

In [13]:
wms.identification.type
wms.identification.version
wms.identification.title
wms.identification.abstract

'The Copernicus project’s Sentinel satellites are revolutionizing earth observation (EO). Its free, full and open access to data with very short revisit times, high spatial resolution, and good spectral resolution are crucial for many applications. The portfolio of possible products is vast - use-cases of such a service range from plant health monitoring, land and water body change, flood monitoring, disaster mapping and more.However the current gap between Sentinel source data and its end-users is large:• \x90  ESA’s complex Scientific Data Hub• \x90  raster files are compressed with JPEG2000 (13 raster filesfor each product, one per spectral band)• \x90  terabytes of data per week• \x90  additional processing requirementsTackling the data in an old-fashioned way -  offering individual derivative products simply does not work anymore, the associated time and costs are large and defeat most of the major benefits of the Sentinel project.Our approach combines cloud-based GIS technologies

In [124]:
list(wms.contents)  #see list of options for types of features available from Sentinel Satelites

['AGRICULTURE',
 'ARI1',
 'ARI2',
 'ATMOSPHERIC_PENETRATION',
 'B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B8A',
 'BAI',
 'BATHYMETRIC',
 'CHL_RED_EDGE',
 'CRI1',
 'CRI2',
 'EVI',
 'EVI2',
 'FALSE_COLOR',
 'FALSE_COLOR_URBAN',
 'GEOLOGY',
 'GRVI1',
 'LAI_SAVI',
 'MOISTURE_INDEX',
 'MSAVI2',
 'NBR_RAW',
 'NDVI',
 'NDVI_GRAY',
 'NDVI_GREEN_GRAY',
 'NDWI',
 'PSRI',
 'PSRI_NIR',
 'RED_EDGE_NDVI',
 'RE_NDWI',
 'RGB_11_8_3',
 'RGB_4_3_1',
 'RGB_8_11_12',
 'RGB_8_11_4',
 'RGB_8_5_4',
 'RGB_8_6_4',
 'SAVI',
 'SWIR',
 'TRUE_COLOR']

In [15]:
wms['TRUE_COLOR'].styles 

{'default': {'title': 'default'}}

In [16]:
[op.name for op in wms.operations]

['GetCapabilities', 'GetMap', 'GetFeatureInfo']

In [17]:
wms.getOperationByName('GetMap').methods

[{'type': 'Get',
  'url': 'http://services.sentinel-hub.com/ogc/wms/02ea2542-7c5b-403a-bcc9-20f8618af983?'}]

In [18]:
wms.getOperationByName('GetFeatureInfo').formatOptions

['application/xml',
 'text/xml',
 'application/vnd.ogc.wms_xml',
 'application/vnd.ogc.gml',
 'text/html',
 'application/json',
 'text/plain']

### Image Retrieval

#### Loop through and download more images

In [19]:
fire_perims = all_fires.copy()

#### General helper functions

In [84]:
from pathlib import Path
import sys

def getSearchIntervalNew(fire_perims,i, isFire):
    """Finds a 10 day time interval matching when fire happened for ith instance of a fire"""
    """For non-fire, check 5 months prior for same location where fire burned"""
    # pull date from fire_perims and reformat it
    #date = fire_perims['DATE'].iloc[i,]
    date = fire_perims['perimeterd'][i]
    
    # Reinterpret the date as a datetime and save as a string

    firedate = dt.datetime.strptime(date, '%Y-%m-%d')
    firedate_str = firedate.strftime("%Y-%m-%d")
    
    if isFire:
        # Calculate the date that is 10 days in the future to be the end of the time interval we want an image
        interval_enddate = firedate + dt.timedelta(days = 5)  #can reduce if needed
        # interval_enddate = interval_enddate.now()
        interval_enddate_str = interval_enddate.strftime("%Y-%m-%d")
        # Construct date interval
        search_interval = firedate_str + '/' + interval_enddate_str 
    else:
        interval_startdate = firedate - dt.timedelta(days = 300)
        interval_enddate = interval_startdate + dt.timedelta(days = 30)
        search_interval = interval_startdate.strftime("%Y-%m-%d") + '/' + interval_enddate.strftime("%Y-%m-%d")  
    
    return (search_interval, str(firedate.strftime("%Y")))


     
        

### Downloading Images as NP_Arrays (New code written on March 4)

#### Sample for one image

In [149]:
i = 0
test_polygon = all_fires['geometry'][0]
poly_center = np.array(test_polygon.representative_point()) 
polygonbounds = getAutomatedBbox(poly_center[0], poly_center[1], distance)

image_request_true = wms.getmap(  
                    layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polygonbounds,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2018-04-05/2019-04-05',        #may 23 (05, 24) to july 1st>             
                     )
image_request_ndvi = wms.getmap(  
                    layers=['NDVI'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2018-04-05/2019-04-20',        #may 23 (05, 24) to july 1st>             
                     )
image_request_nbr = wms.getmap(  
                    layers=['NBR_RAW'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2018-04-05/2019-04-20',        #may 23 (05, 24) to july 1st>             
                     )
                     
    
image_request_bai = wms.getmap(  
                    layers=['BAI'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time ='2018-04-05/2019-04-20',        #may 23 (05, 24) to july 1st>             
                     )


image_array_true = cv2.imdecode(np.frombuffer(image_request_true.read(), np.uint8), -1)
image_array_ndvi = cv2.imdecode(np.frombuffer(image_request_ndvi.read(), np.uint8), -1)
image_array_nbr = cv2.imdecode(np.frombuffer(image_request_nbr.read(), np.uint8), -1)
image_array_nbr = image_array_nbr[..., np.newaxis]
image_array_bai = cv2.imdecode(np.frombuffer(image_request_bai.read(), np.uint8), -1)
image_array_bai = image_array_bai[..., np.newaxis]




image_array_all = np.concatenate((image_array_true, image_array_ndvi, image_array_nbr, image_array_bai), axis=-1)


In [147]:
image_array_all.shape

(256, 256, 8)

###   ALL IMAGES
#### Helper function for getting array For all images (256 x 256 x 12 channels x m examples)

In [109]:
def getDownloadImageArray(polygonbounds, timeparams, date):
    image_request_true = wms.getmap(  
                    layers=['TRUE_COLOR'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polygonbounds,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time =timeparams,                    
                     )
    image_request_ndvi = wms.getmap(  
                    layers=['NDVI'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time =timeparams,                  
                     )
    image_request_nbr = wms.getmap(  
                    layers=['NBR_RAW'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time =timeparams,           
                     )
                     
    
    image_request_bai = wms.getmap(  
                    layers=['BAI'],
                     styles=['default'],
                     srs='EPSG:4326',
                     bbox=polybbox,
                     size=(256, 256),
                     #geometry = ade,
                     format='image/png',
                     #Transparent = False,
                     time =timeparams,                    
                     )


    image_array_true = cv2.imdecode(np.frombuffer(image_request_true.read(), np.uint8), -1)
    image_array_ndvi = cv2.imdecode(np.frombuffer(image_request_ndvi.read(), np.uint8), -1)
    image_array_nbr = cv2.imdecode(np.frombuffer(image_request_nbr.read(), np.uint8), -1)
    #if image_array_nbr is None:
    #    return []

    image_array_nbr = image_array_nbr[..., np.newaxis]
    image_array_bai = cv2.imdecode(np.frombuffer(image_request_bai.read(), np.uint8), -1)
    #if image_array_bai is None:
    #    return []
    image_array_bai = image_array_bai[..., np.newaxis]



    if np.sum(image_array_true) < (256*256*8*255*0.9):    #if the image is not too white
        image_array_all = np.concatenate((image_array_true, image_array_ndvi, image_array_nbr, image_array_bai), axis=-1)
        return image_array_all
    else:
        return []
        
        
        
def downloadAllBurnImagesAsArray(fire_perims, isFire, distance):
    
    #Get the 1st 256 x 256 x 12 channelsarray
    j=0
    l=0
    while l<=0 and j<len(fire_perims):
    
        if type(all_fires['geometry'][j]) == shapely.geometry.polygon.Polygon:
            test_polygon = all_fires['geometry'][j]
            poly_center = np.array(test_polygon.representative_point()) 
            polygonbounds = getAutomatedBbox(poly_center[0], poly_center[1], distance)
        
        elif type(all_fires['geometry'][j]) == shapely.geometry.multipolygon.MultiPolygon:
            test_polygon = all_fires['geometry'][j][0]
            poly_center = np.array(test_polygon.representative_point()) 
            polygonbounds = getAutomatedBbox(poly_center[0], poly_center[1], distance)

        (timeparams, fireDate) =getSearchIntervalNew(fire_perims,j, isFire)

        image_arr = getDownloadImageArray(polygonbounds, timeparams, fireDate)
        l = len(image_arr)
        if l>0:
            image_arr = np.expand_dims(image_arr, axis = 3)    
        j=j+1

    for i in range(j,len(fire_perims)):  #for i in range(len(fire_perims)):

        if type(all_fires['geometry'][i]) == shapely.geometry.polygon.Polygon:
            test_polygon = all_fires['geometry'][i]
            poly_center = np.array(test_polygon.representative_point()) 
            polygonbounds = getAutomatedBbox(poly_center[0], poly_center[1], distance)
        
        elif type(all_fires['geometry'][i]) == shapely.geometry.multipolygon.MultiPolygon:
            test_polygon = all_fires['geometry'][i][0]
            poly_center = np.array(test_polygon.representative_point()) 
            polygonbounds = getAutomatedBbox(poly_center[0], poly_center[1], distance)

        (timeparams, fireDate) =getSearchIntervalNew(fire_perims,i, isFire)
      
        image_i = getDownloadImageArray(polygonbounds, timeparams, fireDate)
        
        
        if len(image_i)>0:
            image_i = np.expand_dims(image_i, axis = 3)           
            image_arr = np.concatenate((image_arr, image_i), axis = -1)
        
    
    return image_arr
                        

## Multi-channel Image Arrays Retrieval

### Class Fire 

In [48]:
isFire = True

In [86]:
distance = 0.5

#Run on small fire array
start = time.time()
fire_perims_small = fire_perims[0:5]  #final image
final_image_array = downloadAllBurnImagesAsArray(fire_perims_small, isFire, distance)
print(time.time() - start)
print(final_image_array.shape)

16.642478227615356
(256, 256, 8, 5)


In [94]:
start = 0
end = 5000
fire_perims_1st5000 = fire_perims[start:end].reset_index()  #final image

final_image_array_1st5000 = downloadAllBurnImagesAsArray(fire_perims_1st5000, isFire, distance)
#np.save('fire_data_1st5000.npy', final_image_array_1st5000) # save


##### Save fire array to file

In [ ]:
filename = 'class_fire_True_data' + str(start) + 'to' + str(end) + '.npy'
np.save(filename, final_image_array_1st5000) 

### Class Non-Fire 

In [119]:
isFire = False
distance = 0.5

In [179]:
#Run on chunks of 5000 again
print(isFire)
start = 0
end = 5000
fire_perims_1st5000 = fire_perims[start:end].reset_index()  #final image

final_image_array_1st5000 = downloadAllBurnImagesAsArray(fire_perims_1st5000, isFire, distance)

#final_image_array_1st5000nofire = downloadAllBurnImagesAsArray(fire_perims_1st5000, isFire, distance)
#np.save('non_fire_data_1st5000.npy', final_image_array_1st5000nofire) # save


False


##### Save non fire array to file

In [2]:
filename = 'class_fire_False_data' + str(start) + 'to' + str(end) + '.npy'
np.save(filename, final_image_array_1st5000) 

#### Load fire and non fire arrays from file

In [3]:
fire_arr = np.load('fire_data.npy') # load
non_fire_arr = np.load('non_fire_data.npy') # load

###  Class Non-fire: Get additional random lat long points of non-fire

In [38]:
def downloadRandomImages(distance, num):
    lat_center = np.random.uniform(35.0000, 45.000)
    lon_center = np.random.uniform(-120.000, -90.000)
        
    polygonbounds = getAutomatedBbox(lon_center, lat_center, distance)
    timeparams = '2018-03-01/2020-03-01'
    date = '2020'
    image_arr = getDownloadImageArray(polygonbounds, timeparams, date)
    image_arr = np.expand_dims(image_arr, axis = 3) 
    for i in range(1,num): 
        random_arr = getDownloadImageArray(polygonbounds, timeparams, date)
        if len(random_arr)>0:
            random_arr = np.expand_dims(random_arr, axis = 3)         
            image_arr = np.concatenate((image_arr, random_arr), axis = -1)
    
    return image_arr


In [39]:
random_array = downloadRandomImages(distance, 50)  # change 50 to however many random images we want.

In [41]:
random_array.shape

In [189]:
isFire = False
print(isFire)
start = 0
end = 1000
num = end - start
random_array = downloadRandomImages(distance, num)  # num is however many random images we want.

False


#### Save random array to file

In [ ]:
filename = 'class_fire_False_random_data' + str(start) + 'to' + str(end) + '.npy'
np.save(filename, random_array)

#### Load random (non-fire) array from file and merge with other nonfire

In [ ]:
random_arr = np.load('randomX_train_val_test.npy') # load

In [ ]:
all_non_fire_arr = np.concatenate((random_arr, non_fire_arr), axis = -1)

#### See shapes of final fire and non-fire arrays

In [ ]:
fire_arr.shape     #This is total X for train_validation_testing split with Y = class fire
all_non_fire_arr.shape #This is total X for train_validation_testing split with Y = class no fire